In [1]:
# importing datasets and merging the 2 datasets on movieId and droping the unwanted columns (genres,timestamp)
import pandas as pd
ratings = pd.read_csv('ContentBasedRecommenderSystem/ratings.csv')
movies = pd.read_csv('ContentBasedRecommenderSystem/movies.csv')
data = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(data.shape)
data.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [2]:
# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data, random_state=42, test_size=0.3)
X_train.shape

(70585, 4)

In [3]:
# Group data by movie id with columns containing user rating and replacing Null values with 0
matrix_movie = X_train.pivot_table(index=['movieId'],columns=['userId'],values='rating').fillna(0)
print(matrix_movie.shape)

(8555, 610)


In [4]:
matrix_movie.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,0.0,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Taking the first 200 users only into consideration
matrix_movie= matrix_movie.iloc[:, 0:200]
matrix_movie.shape

(8555, 200)

In [6]:
matrix_movie.head()

userId,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [7]:
# calculating cosine similarity matrix
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix_movie = cosine_similarity(matrix_movie)
similarity_matrix_movie[np.isnan(similarity_matrix_movie)] = 0
print(similarity_matrix_movie)

[[1.         0.3244358  0.2205925  ... 0.         0.         0.        ]
 [0.3244358  1.         0.15181906 ... 0.         0.         0.        ]
 [0.2205925  0.15181906 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 1.         1.         1.        ]]


In [8]:
# Calculate movie predicted ratings
movie_predicted_ratings = np.dot(similarity_matrix_movie, matrix_movie)
movie_predicted_ratings.shape

(8555, 200)

In [9]:
# Dummy data
dummy_train = X_train.copy()
dummy_test = X_test.copy()

dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x > 0 else 0)

dummy_train_movie = dummy_train.pivot(index = 'movieId', columns = 'userId', values = 'rating').fillna(1)
dummy_train_movie= dummy_train_movie.iloc[:, 0:200]
dummy_test = dummy_test.pivot(index ='movieId', columns = 'userId', values = 'rating').fillna(0)
dummy_train_movie.shape

(8555, 200)

In [10]:
# calculate movie final ratings
movie_final_ratings = np.multiply(movie_predicted_ratings, dummy_train_movie)

In [11]:
from prettytable import PrettyTable
def similar_movies(movie_name):
    # Recommends the top 10 similar movies by sorting descending to given movie name with movie name and prints the movies table
    for k,v in movie_titles.items():
        if v ==movie_name:
            recommendation_movie = movie_final_ratings.iloc[k].sort_values(ascending = False)[0:10]
            myTable = PrettyTable(["Movie ID", "Movie Title"]) 
            for i in recommendation_movie.index:
                if i not in movie_titles:
                    myTable.add_row([i,movie_titles[i+1]])
                else:
                    myTable.add_row([i,movie_titles[i]])  
    print(myTable)



In [44]:
# Print the top 10 similar movies 
movie_titles = dict(zip(movies['movieId'], movies['title']))
similar_movies('Toy Story (1995)')


+----------+---------------------------------------+
| Movie ID |              Movie Title              |
+----------+---------------------------------------+
|    68    |   French Twist (Gazon maudit) (1995)  |
|   182    |          Mute Witness (1994)          |
|   105    | Bridges of Madison County, The (1995) |
|    64    |          Two if by Sea (1996)         |
|    45    |           To Die For (1995)           |
|    91    |           Mary Reilly (1996)          |
|   111    |           Taxi Driver (1976)          |
|   200    |          Three Wishes (1995)          |
|    57    |      Home for the Holidays (1995)     |
|    42    |         Dead Presidents (1995)        |
+----------+---------------------------------------+


In [45]:
similar_movies('Waiting to Exhale (1995)')

+----------+---------------------------------------+
| Movie ID |              Movie Title              |
+----------+---------------------------------------+
|   177    |        Lord of Illusions (1995)       |
|   182    |          Mute Witness (1994)          |
|    64    |          Two if by Sea (1996)         |
|    91    |           Mary Reilly (1996)          |
|   140    |      Up Close and Personal (1996)     |
|    18    |           Four Rooms (1995)           |
|    42    |         Dead Presidents (1995)        |
|    19    | Ace Ventura: When Nature Calls (1995) |
|   105    | Bridges of Madison County, The (1995) |
|    57    |      Home for the Holidays (1995)     |
+----------+---------------------------------------+


In [38]:
# Group data by userid with columns containing user rating of movie and replcae null values with 0
matrix_user = X_train.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)
matrix_user=matrix_user.iloc[0:201, :]
matrix_user.shape

(201, 8555)

In [39]:
# calculating cosine similarity matrix
similarity_matrix_user = cosine_similarity(matrix_user)
similarity_matrix_user[np.isnan(similarity_matrix_user)] = 0

In [40]:
user_predicted_ratings = np.dot(similarity_matrix_user, matrix_user)

In [41]:
# Dummy data
dummy_train = X_train.copy()
dummy_test = X_test.copy()

dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x > 0 else 0)

# The movies not rated by user is marked as 1 for prediction 
dummy_train_user = dummy_train.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(1)
dummy_train_user=dummy_train_user.iloc[0:201, :]
# The movies not rated by user is marked as 0 for evaluation 
dummy_test = dummy_test.pivot(index ='userId', columns = 'movieId', values = 'rating').fillna(0)
dummy_train_user.shape

(201, 8555)

In [42]:
user_final_ratings = np.multiply(user_predicted_ratings, dummy_train_user)

In [43]:
def recommend_for_user(user):
    # Recommends the top 3 similar movies by sorting descending to given UserID and prints the movies table
    for k,v in movie_titles.items():
    recommendation = user_final_ratings.iloc[user].sort_values(ascending = False)[0:3]
    myTable = PrettyTable(["Movie ID", "Movie Title"]) 
    for i in recommendation.index:
        if i not in movie_titles:
            myTable.add_row([i,movie_titles[i+1]])
        else:
            myTable.add_row([i,movie_titles[i]])  

    print(myTable)
recommend_for_user(200)

+----------+-------------------------------------------+
| Movie ID |                Movie Title                |
+----------+-------------------------------------------+
|   318    |      Shawshank Redemption, The (1994)     |
|   110    |             Braveheart (1995)             |
|   260    | Star Wars: Episode IV - A New Hope (1977) |
+----------+-------------------------------------------+
